# 搭建并使用向量数据库

## 前序数据预处理配置

批量处理文件夹中所有文件

In [1]:
import os
# from dotenv import load_dotenv, find_dotenv 
# pip install python-dotenv

# _ = load_dotenv(find_dotenv())


# 获取folder_path下所有文件路径，储存在file_paths里
file_paths = []
folder_path = '../data_base/knowledge_path'
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)
print(file_paths)

['../data_base/knowledge_path\\pumkin_book\\pumpkin_book.pdf']


In [2]:
from langchain.document_loaders.pdf import PyMuPDFLoader
# from langchain.document_loaders.markdown import UnstructuredMarkdownLoader

# 遍历文件路径并把实例化的loader存放在loaders里
loaders = []

for file_path in file_paths:
    file_type = file_path.split('.')[-1]
    if file_type == 'pdf':
        loaders.append(PyMuPDFLoader(file_path))
    else:
        print(f"Unsupported file type: {file_type} for file {file_path}")

In [3]:
# # 下载文件并存储到text
# # 未作数据清洗
# texts = []

# for loader in loaders:
#     texts.extend(loader.load())

# texts[2]

In [4]:
# 下载文件并存储到text
# 作数据清洗
from langchain.document_loaders.pdf import PyMuPDFLoader
import re
# 下载文件并存储到text
texts = []
for loader in loaders:
    texts.extend(loader.load())   

for text in texts:
    pattern = re.compile(r'[^\u4e00-\u9fff](\n)[^\u4e00-\u9fff]', re.DOTALL)
    text.page_content = re.sub(pattern, lambda match: match.group(0).replace('\n', ''), text.page_content)
    text.page_content = text.page_content.replace('•', '')
    text.page_content = text.page_content.replace(' ', '')

In [5]:
texts[0].page_content

'\x01本\x03:1.9.9\n发布日期:2023.03\n南⽠书\nPUMPKINBOOKDatawhale'

In [6]:
# # 下载文件并存储到text
# # 作数据清洗
# from langchain.document_loaders.pdf import PyMuPDFLoader
# import re
# # 下载文件并存储到text
# texts = []
# for loader in loaders:
#     texts.extend(loader.load())   

# for text in texts:
#     text.page_content = re.sub(r'\s*\n\s*', ' ', text.page_content)
#     text.page_content = re.sub(r'[\s•]', '', text.page_content)

载入后的变量类型为`langchain_core.documents.base.Document`, 文档变量类型同样包含两个属性
- `page_content` 包含该文档的内容。
- `meta_data` 为文档相关的描述性数据。

In [7]:
text = texts[100]
print(f"每一个元素的类型：{type(text)}.", 
    # f"该文档的描述性数据：{text.metadata}", 
    f"查看该文档的内容:\n{text.page_content[0:]}", 
    sep="\n------\n")

每一个元素的类型：<class 'langchain_core.documents.base.Document'>.
------
查看该文档的内容:
→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》←_←
当某一个类别j的基分类器的结果之和，大于所有结果之和的12，则选择该类别j为最终结果。8.4.5
式(8.25)的解释
H(x)=cargmaxj
∑Ti=1hji(x)
相比于其他类别，该类别j的基分类器的结果之和最大，则选择类别j为最终结果。8.4.6
式(8.26)的解释
H(x)=cargmaxj
∑Ti=1wihji(x)
相比于其他类别，该类别j的基分类器的结果之和最大，则选择类别j为最终结果，与式(8.25)不同的
是，该式在基分类器前面乘上一个权重系数，该系数大于等于0，且T个权重之和为1。8.4.7
元学习器(meta-learner)的解释
书中第183页最后一行提到了元学习器(meta-learner)，简单解释一下，因为理解meta的含义有时
对于理解论文中的核心思想很有帮助。
元(meta)，非常抽象，例如此处的含义，即次级学习器，或者说基于学习器结果的学习器；另外还有
元语言，就是描述计算机语言的语言，还有元数学，研究数学的数学等等；
另外，论文中经常出现的还有meta-strategy，即元策略或元方法，比如说你的研究问题是多分类问题，
那么你提出了一种方法，例如对输入特征进行变换（或对输出类别做某种变换），然后再基于普通的多分
类方法进行预测，这时你的方法可以看成是一种通用的框架，它虽然针对多分类问题开发，但它需要某个
具体多分类方法配合才能实现，那么这样的方法是一种更高层级的方法，可以称为是一种meta-strategy。8.4.8Stacking算法的解释
该算法其实非常简单，对于数据集，试想你现在有了个基分类器预测结果，也就是说数据集中的每个
样本均有个预测结果，那么怎么结合这个预测结果呢？
本节名为“结合策略”，告诉你各种结合方法，但其实最简单的方法就是基于这个预测结果再进行一
次学习，即针对每个样本，将这个预测结果作为输入特征，类别仍为原来的类别，既然无法抉择如何将这
些结果进行结合，那么就“学习”一下吧。“西瓜书”图8.9伪代码第9行中将第个样本进行变换，特征为个基学习器的输出，类别标记仍为原
来的，将所

In [8]:
''' 
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''
#导入文本分割器
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
# 知识库中单段文本长度
CHUNK_SIZE = 512

# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

In [10]:
# 使用递归字符文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
# text_splitter.split_text(text.page_content[0:1000])

In [11]:
split_docs = text_splitter.split_documents(texts)
print(f"切分后的文件数量：{len(split_docs)}")

切分后的文件数量：614


In [12]:
print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")

切分后的字符数（可以用来大致评估 token 数）：252076


In [13]:
split_docs[2].page_content

'有点飘的时候再回来啃都来得及；每个公式的解析和推导我们都力(zhi)争(neng)以本科数学基础的视角进行讲解，所以超纲的数学知识\n我们通常都会以附录和参考文献的形式给出，感兴趣的同学可以继续沿着我们给的资料进行深入学习；若南瓜书里没有你想要查阅的公式，或者你发现南瓜书哪个地方有错误，请毫不犹豫地去我们GitHub的\nIssues（地址：https://github.com/datawhalechina/pumpkin-book/issues）进行反馈，在对应版块\n提交你希望补充的公式编号或者勘误信息，我们通常会在24小时以内给您回复，超过24小时未回复的\n话可以微信联系我们（微信号：at-Sm1les）；\n配套视频教程：https://www.bilibili.com/video/BV1Mh411e7VU\n在线阅读地址：https://datawhalechina.github.io/pumpkin-book（仅供第1版）\n最新版PDF获取地址：https://github.com/datawhalechina/pumpkin-book/releases\n编委会'

## 构建Chroma向量库

### 构建Chroma向量库-embedding - zhipu

Langchain 集成了超过 30 个不同的向量存储库。我们选择 Chroma 是因为它轻量级且数据存储在内存中，这使得它非常容易启动和开始使用。

测试API调用是否成功

In [14]:
# 测试
from zhipuai import ZhipuAI

client = ZhipuAI(api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io") 
response = client.embeddings.create(
    model="embedding-2", #填写需要调用的模型编码
     input=[
        "美食非常美味，服务员也很友好。",
        "这部电影既刺激又令人兴奋。",
        "阅读书籍是扩展知识的好方法。"
    ],
)
# print(response)

In [15]:
# langchain_chroma.vectorstores.Chroma
class langchain_chroma.vectorstores.Chroma(
    collection_name: str = 'langchain', 
embedding_function: Embeddings | None = None, 
persist_directory: str | None = None, 
client_settings: Settings | None = None, 
collection_metadata: Dict | None = None, 
client: ClientAPI | None = None, 
relevance_score_fn: Callable[[float], float] | None = None, 
create_collection_if_not_exists: bool | None = True)


classmethod from_documents(
    documents: List[Document], 
embedding: Embeddings | None = None, 
ids: List[str] | None = None, 
collection_name: str = 'langchain', 
persist_directory: str | None = None, 
client_settings: Settings | None = None, 
client: ClientAPI | None = None, 
collection_metadata: Dict | None = None, 
**kwargs: Any)

SyntaxError: invalid syntax (306647126.py, line 2)

直接调用Chroma.from_documents

In [17]:
# 汇总

import os
from langchain.vectorstores.chroma import Chroma
from langchain_community.embeddings import ZhipuAIEmbeddings

# 定义持久化目录
persist_directory = '../chromatest'

# 创建嵌入模型
embedding = ZhipuAIEmbeddings(model="embedding-2", api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io")

try:
    # 初始化 Chroma 向量数据库
    vectordb = Chroma.from_documents(
        documents=split_docs[:20],  # 为了速度，只选择前 20 个切分的 doc 进行生成
        embedding=embedding,
        # collection_name="test1",
        persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
    )
    
    # 持久化向量数据库
    vectordb.persist()
    print("向量数据库已成功持久化到磁盘。")
except Exception as e:
    print(f"持久化过程中发生错误: {e}")

持久化过程中发生错误: The onnxruntime python package is not installed. Please install it with `pip install onnxruntime`


In [18]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions

chroma_client = chromadb.HttpClient(host='localhost', port=8000)

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io",
                model_name="embedding-2"
            )
collection = chroma_client.get_or_create_collection(
    name="test4",
    embedding_function=openai_ef,  # 使用openai
    metadata={
        "hnsw:space": "cosine", 
        "hnsw:construction_ef": 150,
        "hnsw:M": 24,
        "hnsw:num_threads": 8
    }
)
ids_list = [f"doc_{i+1}" for i in range(len(split_docs[:20]))]
print(ids_list)

try:
    # 添加文档到集合
    collection.add(documents=split_docs[:20], ids =ids_list )
    # 持久化向量数据库
    vectordb.persist()
    print("向量数据库已成功持久化到磁盘。")
except Exception as e:
    print(f"持久化过程中发生错误: {e}")

ValueError: The onnxruntime python package is not installed. Please install it with `pip install onnxruntime`

通过chroma数据库add

In [87]:
from chromadb.utils import embedding_functions
import os
from langchain.vectorstores.chroma import Chroma
from langchain_community.embeddings import ZhipuAIEmbeddings
from chromadb import PersistentClient
# 定义持久化目录
persist_directory = './chroma6'
# 初始化客户端（持久化模式）
client = PersistentClient(path=persist_directory)

import chromadb.utils.embedding_functions as embedding_functions
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io",
                model_name="embedding-2"
            )
collection = client.get_or_create_collection(
    name="test4",
    embedding_function=openai_ef,  # 使用openai
    metadata={
        "hnsw:space": "cosine", 
        "hnsw:construction_ef": 150,
        "hnsw:M": 24,
        "hnsw:num_threads": 8
    }
)
ids_list = [f"doc_{i+1}" for i in range(len(split_docs[:20]))]
print(ids_list)

try:
    # 添加文档到集合
    collection.add(documents=split_docs[:20], ids =ids_list )
    # 持久化向量数据库
    vectordb.persist()
    print("向量数据库已成功持久化到磁盘。")
except Exception as e:
    print(f"持久化过程中发生错误: {e}")

['doc_1', 'doc_2', 'doc_3', 'doc_4', 'doc_5', 'doc_6', 'doc_7', 'doc_8', 'doc_9', 'doc_10', 'doc_11', 'doc_12', 'doc_13', 'doc_14', 'doc_15', 'doc_16', 'doc_17', 'doc_18', 'doc_19', 'doc_20']
持久化过程中发生错误: Expected document to be a str, got page_content='本:1.9.9
发布日期:2023.03
南⽠书
PUMPKINBOOKDatawhale' metadata={'producer': 'xdvipdfmx (20200315)', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-03-03T17:07:09+00:00', 'source': '../../data_base/knowledge_path\\pumkin_book\\pumpkin_book.pdf', 'file_path': '../../data_base/knowledge_path\\pumkin_book\\pumpkin_book.pdf', 'total_pages': 196, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20230303170709-00'00'", 'page': 0} in add.


通过chroma数据库add-改良版 每次三个doc，分批次导入

In [88]:
# 汇总

import os
from langchain.vectorstores.chroma import Chroma
from langchain_community.embeddings import ZhipuAIEmbeddings
from chromadb import PersistentClient
from chromadb import Documents, EmbeddingFunction, Embeddings
# 定义持久化目录
persist_directory = './chroma6'
# 初始化客户端（持久化模式）
client = PersistentClient(path=persist_directory)

# 创建嵌入模型
import chromadb.utils.embedding_functions as embedding_functions
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io",
                model_name="embedding-2"
            )

# zhipu_ef = ZhipuAIAdapter(model_name="embedding-2", api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io")

# 创建集合并配置HNSW参数
# collection = client.create_collection( 
collection = client.get_or_create_collection(
    name="test3",
    embedding_function=openai_ef,
    metadata={
        "hnsw:space": "cosine", 
        "hnsw:construction_ef": 150,
        "hnsw:M": 24,
        "hnsw:num_threads": 8
    }
)

ids_list = [f"doc_{i+1}" for i in range(len(split_docs[:20]))]
print(ids_list)

try:
    # 添加文档到集合
    # collection.add(documents=split_docs)

    from itertools import batched
    docs_batches = batched(split_docs[:20], 3)  # 分3个批次
    ids_batches = batched(ids_list, 3)

    for docs, ids in zip(docs_batches, ids_batches):
        collection.add(documents=docs, ids=ids)
    
    # 持久化向量数据库
    vectordb.persist()
    print("向量数据库已成功持久化到磁盘。")
except Exception as e:
    print(f"持久化过程中发生错误: {e}")

['doc_1', 'doc_2', 'doc_3', 'doc_4', 'doc_5', 'doc_6', 'doc_7', 'doc_8', 'doc_9', 'doc_10', 'doc_11', 'doc_12', 'doc_13', 'doc_14', 'doc_15', 'doc_16', 'doc_17', 'doc_18', 'doc_19', 'doc_20']
持久化过程中发生错误: Expected document to be a str, got (Document(metadata={'producer': 'xdvipdfmx (20200315)', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-03-03T17:07:09+00:00', 'source': '../../data_base/knowledge_path\\pumkin_book\\pumpkin_book.pdf', 'file_path': '../../data_base/knowledge_path\\pumkin_book\\pumpkin_book.pdf', 'total_pages': 196, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20230303170709-00'00'", 'page': 0}, page_content='\x01本\x03:1.9.9\n发布日期:2023.03\n南⽠书\nPUMPKINBOOKDatawhale'), Document(metadata={'producer': 'xdvipdfmx (20200315)', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-03-03T17:07:09+00:00', 'source': '../../data_base/knowledge_path\\pumkin_book\\pumpkin_bo

In [15]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

NameError: name 'vectordb' is not defined

连接chroma数据库

In [90]:
# 数据库路径 ./chroma2

In [26]:
import chromadb
chroma_client  = chromadb.PersistentClient(path="../chromatest")

ValueError: The onnxruntime python package is not installed. Please install it with `pip install onnxruntime`

In [ ]:
chroma_client.list_collections()

In [54]:
# 获取一个存在的Collection对象
collection = client.get_collection("langchain")

In [55]:
collection.count()  #  returns the number of items in the collection.

20

In [56]:
# collection.peek()  # returns a list of the first 10 items in the collection.

{'ids': ['89242ee3-4fbc-4071-8114-eb9afe70e9a9',
  'b06a60d3-dad2-448e-b386-6a3c729274c9',
  '76220418-63d5-4c18-b6e0-8517b9dc27b6',
  '8b6ac177-f1bc-4681-8bed-d24d927010b9',
  '20f5a795-4014-43c4-9acb-6930e4bb49c6',
  'a9b7ac2e-51f0-4b8f-9951-f9be922929cb',
  '4e71b8b4-77c8-4c5a-a633-22eafcdc0011',
  '13f4e2b5-951e-4a9d-9e82-2aad3ee07c50',
  '0ed2214c-8d03-4a10-b502-a8644774a5c2',
  '13f4b2ca-96ec-43df-ad96-c84206de1ab9'],
 'embeddings': array([[-2.27248501e-02,  2.65129711e-02, -1.01085668e-02, ...,
          2.21525766e-02, -2.63613071e-02,  4.97833965e-03],
        [-1.21111041e-02, -9.84291732e-03,  1.38132088e-03, ...,
         -2.29995698e-02, -5.22771059e-03, -1.91159907e-03],
        [ 4.15145196e-02,  1.10516725e-02,  2.18729302e-02, ...,
          1.87054705e-02,  3.80822155e-03,  1.18502639e-02],
        ...,
        [ 6.88662454e-02,  4.38260138e-02,  1.40380645e-02, ...,
          1.35815144e-02,  2.90697366e-02,  1.46892639e-02],
        [ 8.09496716e-02,  4.33344357e-02

In [57]:
# 获取所有数据（默认不返回嵌入向量）
all_data = collection.get()
print(all_data)


{'ids': ['89242ee3-4fbc-4071-8114-eb9afe70e9a9', 'b06a60d3-dad2-448e-b386-6a3c729274c9', '76220418-63d5-4c18-b6e0-8517b9dc27b6', '8b6ac177-f1bc-4681-8bed-d24d927010b9', '20f5a795-4014-43c4-9acb-6930e4bb49c6', 'a9b7ac2e-51f0-4b8f-9951-f9be922929cb', '4e71b8b4-77c8-4c5a-a633-22eafcdc0011', '13f4e2b5-951e-4a9d-9e82-2aad3ee07c50', '0ed2214c-8d03-4a10-b502-a8644774a5c2', '13f4b2ca-96ec-43df-ad96-c84206de1ab9', '69e0f7e5-c398-4290-b9a0-4c3616fa9e79', '74908d1d-f5c3-4a42-b10d-aa7a58ab0f37', '73e9183e-f94a-4b2f-b9ca-cece0760d324', '9907c679-de70-463d-953f-dd5176512e50', 'd60d1292-8b0c-4ea5-8154-b521df3a57da', 'd3e23d70-ca36-4b2a-98c8-49161ea53cea', 'e050df56-0115-40e9-bf3a-351d01c1dabb', '4b9235d1-9f75-4597-9aa5-6ef0fa15d632', 'c9f6d7eb-7dcf-4885-9383-8683a191be9f', 'c77cdfd6-d936-47f5-a1aa-3ca80a241654'], 'embeddings': None, 'documents': ['\x01本\x03:1.9.9\n发布日期:2023.03\n南⽠书\nPUMPKINBOOKDatawhale', '前言\n“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读\n者通过西瓜书对机器学习有所了解,所以在书中对部分公式的推导细节没有详述，但是这对那些

## 三、向量检索
### 3.1 相似度检索
Chroma的相似度搜索使用的是余弦距离，即：
$$
similarity = cos(A, B) = \frac{A \cdot B}{\parallel A \parallel \parallel B \parallel} = \frac{\sum_1^n a_i b_i}{\sqrt{\sum_1^n a_i^2}\sqrt{\sum_1^n b_i^2}}
$$
其中$a_i$、$b_i$分别是向量$A$、$B$的分量。

当你需要数据库返回严谨的按余弦相似度排序的结果时可以使用`similarity_search`函数。

In [ ]:
question="什么是南瓜书"

In [ ]:
sim_docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(sim_docs)}")

In [ ]:
for i, sim_doc in enumerate(sim_docs):
    print(f"检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

### 3.2 MMR检索
如果只考虑检索出内容的相关性会导致内容过于单一，可能丢失重要信息。

最大边际相关性 (`MMR, Maximum marginal relevance`) 可以帮助我们在保持相关性的同时，增加内容的丰富度。

核心思想是在已经选择了一个相关性高的文档之后，再选择一个与已选文档相关性较低但是信息丰富的文档。这样可以在保持相关性的同时，增加内容的多样性，避免过于单一的结果。

In [ ]:
mmr_docs = vectordb.max_marginal_relevance_search(question,k=3)

In [ ]:
for i, sim_doc in enumerate(mmr_docs):
    print(f"MMR 检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

# rerank

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

# Cohere Rerank配置
import cohere
cohere_client = cohere.Client(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ")
compressor = CohereRerank(
    client=cohere_client,
    top_n=5  # 保留Top5相关文档
)

# 创建带重排序的检索器
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

# 5. LLM模型初始化
from langchain_community.llms import ChatGLM

# 加载ChatGLM模型（需本地部署）
llm = ChatGLM(
    endpoint_url="http://localhost:8000/v1/chat/completions",
    max_tokens=4096,
    temperature=0.1
)

# 6. 构建RAG链
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 提示模板设计
template = """基于以下上下文，用中文回答用户问题：
上下文：{context}
问题：{question}
要求：答案需包含引用来源的页码（示例：[P3]）"""
prompt = ChatPromptTemplate.from_template(template)

# 组装RAG链
rag_chain = (
    {"context": compression_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

# 7. 使用示例
response = rag_chain.invoke("请解释量子计算的核心原理？")
print(response)